In [ ]:

pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=ba45877f2768c24139101e89c0a1234eb32be7d873bbe3b4ac285978b6de0d17
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import udf, col,when
import numpy as np
from IPython.display import Image
from IPython.display import display


In [ ]:

spark = SparkSession.builder.appName("recommendation").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

data = spark.read.csv('/content/drive/MyDrive/PySpark_BigData_Movie_Recommendations/DataSet/ml-25m/ml-25m/ratings.csv',inferSchema=True,header=True)

data.show()
data.head()
data.printSchema()
data.describe().show()

movies_df = spark.read.csv("/content/drive/MyDrive/PySpark_BigData_Movie_Recommendations/DataSet/ml-25m/ml-25m/movies.csv",inferSchema=True,header=True)
movies_df.printSchema()
movies_df.show(1)

links_df = spark.read.csv("/content/drive/MyDrive/PySpark_BigData_Movie_Recommendations/DataSet/ml-25m/ml-25m/links.csv",inferSchema=True,header=True)
links_df.printSchema()



/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timesta

In [ ]:
# Smaller dataset so we will use 0.8 / 0.2
(train_data, test_data) = data.randomSplit([0.8, 0.2], seed=42)

# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(train_data)


In [ ]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test_data)
new_predictions = predictions.filter(col('prediction') != np.nan)

# Create the evaluator
evaluator = RegressionEvaluator(labelCol="rating", predictionCol="prediction", metricName="rmse")

# Calculate the RMSE
rmse = evaluator.evaluate(new_predictions)
# Print the RMSE as a float value
print(f"RMSE: {rmse:.2f}")

RMSE: 0.81


In [ ]:
predictions.show(n=10)

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|     1|    307|   5.0|1147868828|  4.049979|
|     1|   1175|   3.5|1147868826| 3.8243296|
|     1|   1237|   5.0|1147868839| 3.7773128|
|     1|   2012|   2.5|1147868068| 2.7767134|
|     1|   2692|   5.0|1147869100| 3.7013233|
|     1|   3949|   5.0|1147868678| 3.4603405|
|     1|   4973|   4.5|1147869080|  4.477309|
|     1|   5912|   3.0|1147878698| 2.9157495|
|     1|   7318|   2.0|1147879850| 1.8171695|
|     1|   7323|   3.5|1147869119|  3.926831|
+------+-------+------+----------+----------+
only showing top 10 rows



In [ ]:
predictions.join(movies_df, "movieId").select("userId", "title", "genres", "prediction").show(5)

+------+--------------------+--------------------+----------+
|userId|               title|              genres|prediction|
+------+--------------------+--------------------+----------+
|     1|Three Colors: Blu...|               Drama|  4.049979|
|     1| Delicatessen (1991)|Comedy|Drama|Romance| 3.8243296|
|     1|Seventh Seal, The...|               Drama| 3.7773128|
|     1|Back to the Futur...|Adventure|Comedy|...| 2.7767134|
|     1|Run Lola Run (Lol...|        Action|Crime| 3.7013233|
+------+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
# Prediction for the user with userId 25
for_one_user = predictions.filter(col("userId") == 28).join(movies_df, "movieId").join(links_df, "movieId").select("userId", "title", "genres", "prediction", "tmdbId")
for_one_user.show(5)


+------+--------------------+--------------------+----------+------+
|userId|               title|              genres|prediction|tmdbId|
+------+--------------------+--------------------+----------+------+
|    28|Star Wars: Episod...|Action|Adventure|...| 4.9823318|    11|
|    28|Mystery Science T...|       Comedy|Sci-Fi|  4.646061|  3065|
|    28|E.T. the Extra-Te...|Children|Drama|Sc...| 3.7027588|   601|
|    28|        Alien (1979)|       Horror|Sci-Fi|  5.272458|   348|
|    28| Lost Highway (1997)|Crime|Drama|Fanta...|  4.375308|   638|
+------+--------------------+--------------------+----------+------+
only showing top 5 rows



In [ ]:
import webbrowser

link = "https://www.themoviedb.org/movie/"
for movie in for_one_user.take(2):
    movieURL = link + str(movie.tmdbId)
    print(movie.title)
    print(movieURL)
    webbrowser.open(movieURL)


Star Wars: Episode IV - A New Hope (1977)
https://www.themoviedb.org/movie/11
Mystery Science Theater 3000: The Movie (1996)
https://www.themoviedb.org/movie/3065


In [ ]:
# Generate top 5 movie recommention for each user
userRecomments = model.recommendForAllUsers(5)
# Generate top 5 user recommention for each movei
movieRecomments = model.recommendForAllItems(5)

In [ ]:
movieRecomments.select("movieId", "recommendations.userId").show(10, False)


In [ ]:
users = data.select("userId").distinct().limit(3);
users.show()


In [ ]:
userSubsetRecs = model.recommendForUserSubset(users, 10)


In [ ]:

userSubsetRecs.show()

In [ ]:
userSubsetRecs.select("userId", "recommendations.movieId").show(10, False)

In [ ]:
# Generate top 10 user recommendation for specied set fo Movie
movies = data.select("movieId").distinct().limit(3)
movies.show()

In [ ]:
movieSubSetRecs = model.recommendForItemSubset(movies, 10)
movieSubSetRecs.select("movieId", "recommendations.userId").show(10, False)

In [ ]:
# Prediction for some movies for specified user
movie_ids = [1580, 3175, 2366, 1590]
user_id = 471
# Create a DataFrame with the specified user ID and movie IDs
new_user_preds = spark.createDataFrame([(user_id, movie_id) for movie_id in movie_ids], schema=['userId', 'movieId'])
# Generate predictions for the specified user and movies
new_predictions = model.transform(new_user_preds)
# Show the predictions
new_predictions.show()




+------+-------+----------+
|userId|movieId|prediction|
+------+-------+----------+
|   471|   1580| 3.5902014|
|   471|   3175|  3.307518|
|   471|   2366| 2.0790517|
|   471|   1590| 3.2519007|
+------+-------+----------+



In [ ]:
new_predictions = model.transform(new_user_preds)
new_predictions.show()

+------+-------+----------+
|userId|movieId|prediction|
+------+-------+----------+
|   471|   1580| 3.5902014|
|   471|   3175|  3.307518|
|   471|   2366| 2.0790517|
|   471|   1590| 3.2519007|
+------+-------+----------+



In [ ]:
single_user = test_data.filter(test_data['userId']==12).select(['movieId','userId'])

# User had 10 ratings in the test data set
# Realistically this should be some sort of hold out set!
single_user.show()

reccomendations = model.transform(single_user)

reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+
|movieId|userId|
+-------+------+
|      1|    12|
|     50|    12|
|     88|    12|
|    101|    12|
|    140|    12|
|    163|    12|
|    203|    12|
|    209|    12|
|    257|    12|
|    319|    12|
|    351|    12|
|    377|    12|
|    435|    12|
|    441|    12|
|    442|    12|
|    471|    12|
|    489|    12|
|    497|    12|
|    508|    12|
|    524|    12|
+-------+------+
only showing top 20 rows

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|    745|    12|  4.372803|
|   4973|    12|  4.323984|
|   1136|    12| 4.2682495|
|   1212|    12| 4.2402496|
|    926|    12|  4.191748|
|    858|    12|  4.170047|
|     50|    12|  4.166792|
|   3996|    12|  4.119478|
|  74416|    12| 4.1137567|
|    527|    12|  4.106221|
|   1361|    12| 4.0702014|
|   1225|    12|  4.038475|
|  40870|    12|  4.034732|
|  63876|    12| 4.0299177|
|   1175|    12| 4.0199904|
|   1219|    12| 4.0198536|
|   2788|    12| 3.9846127|
|   12

In [ ]:
# User had 10 ratings in the test data set
# Realistically this should be some sort of hold out set!
single_user = test_data.filter(test_data['userId'] == 12).select(['movieId', 'userId'])

# Show the user's ratings in the test data set
single_user.show()

# Generate recommendations for the specified user
recommendations = model.transform(single_user)

# Join movie names to the recommendations
recommendations_with_movie_name = recommendations.join(movies_df, "movieId").select(
    "userId", "title", "genres", "prediction"
)

# Show the recommendations sorted by prediction in descending order
recommendations_with_movie_name.orderBy('prediction', ascending=False).show()

+-------+------+
|movieId|userId|
+-------+------+
|      1|    12|
|     50|    12|
|     88|    12|
|    101|    12|
|    140|    12|
|    163|    12|
|    203|    12|
|    209|    12|
|    257|    12|
|    319|    12|
|    351|    12|
|    377|    12|
|    435|    12|
|    441|    12|
|    442|    12|
|    471|    12|
|    489|    12|
|    497|    12|
|    508|    12|
|    524|    12|
+-------+------+
only showing top 20 rows

+------+--------------------+--------------------+----------+
|userId|               title|              genres|prediction|
+------+--------------------+--------------------+----------+
|    12|Wallace & Gromit:...|Animation|Childre...|  4.372803|
|    12|Amelie (Fabuleux ...|      Comedy|Romance|  4.323984|
|    12|Monty Python and ...|Adventure|Comedy|...| 4.2682495|
|    12|Third Man, The (1...|Film-Noir|Mystery...| 4.2402496|
|    12|All About Eve (1950)|               Drama|  4.191748|
|    12|Godfather, The (1...|         Crime|Drama|  4.170047|
|    12|